# 🎯 Day 1: Clustering Algorithms

**🎯 Goal:** Master clustering techniques to find hidden patterns in unlabeled data

**⏱️ Time:** 60-75 minutes

**🌟 Why This Matters for AI:**
- Essential for RAG systems: cluster similar documents for efficient retrieval
- Customer segmentation: group users by behavior without predefined categories
- Used by OpenAI to organize knowledge bases, by Meta for content grouping
- Critical for multimodal AI: cluster images, text, and embeddings together
- Powers recommendation systems, anomaly detection, and data exploration

---

## 🤔 What is Clustering?

**Clustering** = Finding groups (clusters) in data where:
- Items in the same cluster are **similar** to each other
- Items in different clusters are **different** from each other

**Key difference from classification:**
- Classification (supervised): You have labels → "This is a cat, this is a dog"
- Clustering (unsupervised): No labels → "Find natural groups in this data"

**Real-world example:**
Imagine you have 10,000 customer purchase histories but no categories. Clustering can discover:
- Cluster 1: Budget shoppers (frequent small purchases)
- Cluster 2: Premium buyers (rare expensive purchases)
- Cluster 3: Seasonal shoppers (holiday spikes)

Let's master the 3 most important clustering algorithms! 👇

In [ ]:
# Import our tools
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.datasets import make_blobs, make_moons
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import silhouette_score, davies_bouldin_score

# Set random seed and style
np.random.seed(42)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ Libraries imported successfully!")
print("📊 Ready to explore clustering algorithms!")

## 🎯 Algorithm 1: K-Means Clustering

**K-Means** = The most popular clustering algorithm!

### How it works:
1. **Choose K** (number of clusters you want)
2. **Initialize**: Randomly place K cluster centers (centroids)
3. **Assign**: Assign each point to the nearest centroid
4. **Update**: Move centroids to the center of their assigned points
5. **Repeat** steps 3-4 until centroids stop moving

### Strengths:
- ✅ Fast and scalable (works on millions of points)
- ✅ Simple to understand and implement
- ✅ Guaranteed to converge

### Weaknesses:
- ❌ You must specify K in advance
- ❌ Sensitive to initial centroid placement
- ❌ Assumes spherical clusters (equal variance)
- ❌ Struggles with non-circular shapes

### Real AI Use (2024-2025):
- **RAG systems:** Cluster document embeddings for faster retrieval
- **Vector databases:** Organize embeddings (Pinecone, Weaviate use variants)
- **Image compression:** Reduce colors by clustering pixel values
- **Customer segmentation:** Group users by behavior patterns

In [ ]:
# Create sample data: customer shopping patterns
# Features: [average_purchase_amount, purchase_frequency_per_month]
X_customers, true_labels = make_blobs(
    n_samples=300,
    n_features=2,
    centers=4,
    cluster_std=0.8,
    random_state=42
)

# Apply K-Means with K=4 clusters
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_customers)
centroids = kmeans.cluster_centers_

# Visualize the results
plt.figure(figsize=(14, 6))

# Plot 1: Before clustering (raw data)
plt.subplot(1, 2, 1)
plt.scatter(X_customers[:, 0], X_customers[:, 1], c='gray', alpha=0.6, s=50)
plt.title('Before Clustering: Raw Customer Data', fontsize=14, fontweight='bold')
plt.xlabel('Average Purchase Amount ($)', fontsize=11)
plt.ylabel('Purchase Frequency (per month)', fontsize=11)
plt.grid(alpha=0.3)

# Plot 2: After K-Means clustering
plt.subplot(1, 2, 2)
scatter = plt.scatter(X_customers[:, 0], X_customers[:, 1], 
                     c=cluster_labels, cmap='viridis', alpha=0.6, s=50)
plt.scatter(centroids[:, 0], centroids[:, 1], 
           c='red', marker='X', s=300, edgecolors='black', linewidths=2,
           label='Centroids')
plt.title('After K-Means: Discovered Customer Segments', fontsize=14, fontweight='bold')
plt.xlabel('Average Purchase Amount ($)', fontsize=11)
plt.ylabel('Purchase Frequency (per month)', fontsize=11)
plt.legend()
plt.colorbar(scatter, label='Cluster ID')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("🎯 K-Means Results:")
print(f"   - Found {kmeans.n_clusters} customer segments")
print(f"   - Converged in {kmeans.n_iter_} iterations")
print(f"   - Inertia (sum of squared distances): {kmeans.inertia_:.2f}")
print("\n💡 Interpretation:")
print("   Red X marks = Cluster centers (typical customer profile for each segment)")
print("   Each color = Different customer segment discovered automatically!")

### 🔍 Finding the Optimal K: The Elbow Method

**Problem:** How do we choose K?

**Solution:** The Elbow Method!
- Try different values of K (e.g., 1 to 10)
- Plot the "inertia" (sum of squared distances to nearest centroid)
- Look for the "elbow" where adding more clusters doesn't help much

**Inertia:**
- Lower = Better (points are closer to their centroids)
- But it always decreases as K increases (K=N gives inertia=0)
- We want the "sweet spot" where improvement slows down

In [ ]:
# Try K values from 1 to 10
k_values = range(1, 11)
inertias = []
silhouette_scores = []

for k in k_values:
    kmeans_temp = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans_temp.fit(X_customers)
    inertias.append(kmeans_temp.inertia_)
    
    # Silhouette score (only for k > 1)
    if k > 1:
        score = silhouette_score(X_customers, kmeans_temp.labels_)
        silhouette_scores.append(score)
    else:
        silhouette_scores.append(0)

# Plot both metrics
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Elbow method
axes[0].plot(k_values, inertias, marker='o', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Clusters (K)', fontsize=12)
axes[0].set_ylabel('Inertia', fontsize=12)
axes[0].set_title('Elbow Method: Finding Optimal K', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)
axes[0].axvline(x=4, color='red', linestyle='--', alpha=0.7, label='Elbow at K=4')
axes[0].legend()

# Plot 2: Silhouette scores
axes[1].plot(k_values, silhouette_scores, marker='s', color='green', linewidth=2, markersize=8)
axes[1].set_xlabel('Number of Clusters (K)', fontsize=12)
axes[1].set_ylabel('Silhouette Score', fontsize=12)
axes[1].set_title('Silhouette Score: Cluster Quality', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)
axes[1].axhline(y=0.5, color='orange', linestyle='--', alpha=0.7, label='Good threshold (0.5)')
axes[1].legend()

plt.tight_layout()
plt.show()

print("📊 How to Read These Charts:")
print("\n🔵 Elbow Method (Left):")
print("   - Look for the 'elbow' where the curve bends")
print("   - After the elbow, adding clusters gives diminishing returns")
print("   - Here: K=4 seems optimal (elbow point)")
print("\n🟢 Silhouette Score (Right):")
print("   - Range: -1 to 1 (higher is better)")
print("   - > 0.5 = Good clustering")
print("   - > 0.7 = Excellent clustering")
print(f"   - Best score: K={k_values[np.argmax(silhouette_scores)]} (score: {max(silhouette_scores):.3f})")

## 🌳 Algorithm 2: Hierarchical Clustering

**Hierarchical Clustering** = Build a tree (dendrogram) of clusters!

### Two approaches:
1. **Agglomerative (Bottom-up):** Start with each point as a cluster, merge closest pairs
2. **Divisive (Top-down):** Start with one cluster, split recursively

We'll use **Agglomerative** (most common).

### How it works:
1. Start: Each point is its own cluster (N clusters for N points)
2. Find the two closest clusters and merge them
3. Repeat until you have K clusters (or just 1)

### Linkage methods (how to measure "distance" between clusters):
- **Single:** Closest points between clusters
- **Complete:** Farthest points between clusters
- **Average:** Average distance of all pairs
- **Ward:** Minimize variance when merging (most popular)

### Strengths:
- ✅ Don't need to specify K in advance
- ✅ Produces a dendrogram (visual hierarchy)
- ✅ Deterministic (same result every time)
- ✅ Can handle any distance metric

### Weaknesses:
- ❌ Slow for large datasets (O(N²) or O(N³))
- ❌ Can't undo a merge (greedy algorithm)

### Real AI Use:
- **Document clustering:** Organize papers/articles into topic hierarchies
- **Genomics:** Cluster genes by expression patterns
- **Taxonomy:** Create hierarchical categories

In [ ]:
# Create smaller dataset for visualization (hierarchical is slower)
X_small, _ = make_blobs(n_samples=50, n_features=2, centers=3, 
                        cluster_std=0.6, random_state=42)

# Apply Agglomerative Clustering
hierarchical = AgglomerativeClustering(n_clusters=3, linkage='ward')
hierarchical_labels = hierarchical.fit_predict(X_small)

# Create linkage matrix for dendrogram
linkage_matrix = linkage(X_small, method='ward')

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Dendrogram (tree structure)
dendrogram(linkage_matrix, ax=axes[0], color_threshold=10)
axes[0].set_title('Dendrogram: Hierarchical Cluster Tree', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Sample Index', fontsize=11)
axes[0].set_ylabel('Distance (Ward Linkage)', fontsize=11)
axes[0].axhline(y=10, color='red', linestyle='--', label='Cut at 3 clusters')
axes[0].legend()

# Plot 2: Resulting clusters
scatter = axes[1].scatter(X_small[:, 0], X_small[:, 1], 
                         c=hierarchical_labels, cmap='viridis', 
                         s=100, alpha=0.7, edgecolors='black')
axes[1].set_title('Hierarchical Clustering Result', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Feature 1', fontsize=11)
axes[1].set_ylabel('Feature 2', fontsize=11)
plt.colorbar(scatter, ax=axes[1], label='Cluster ID')

plt.tight_layout()
plt.show()

print("🌳 Hierarchical Clustering Results:")
print(f"   - Created {hierarchical.n_clusters_} clusters")
print("\n📊 How to Read the Dendrogram (Left):")
print("   - Bottom: Individual points")
print("   - Vertical lines: Merges between clusters")
print("   - Height: Distance when clusters merged")
print("   - Red horizontal line: Where we 'cut' the tree to get 3 clusters")
print("\n💡 Advantage: You can see the FULL hierarchy, then choose K afterward!")

## 🎯 Algorithm 3: DBSCAN (Density-Based Clustering)

**DBSCAN** = Density-Based Spatial Clustering of Applications with Noise

**Core idea:** Clusters are dense regions separated by sparse regions.

### Key parameters:
- **eps (ε):** Maximum distance between two points to be neighbors
- **min_samples:** Minimum points in a neighborhood to form a dense region

### Point types:
1. **Core point:** Has ≥ min_samples points within eps distance
2. **Border point:** Within eps of a core point, but not core itself
3. **Noise point:** Neither core nor border (outlier!)

### How it works:
1. Pick a random point
2. Find all points within eps distance
3. If ≥ min_samples neighbors → core point → start a cluster
4. Expand cluster by adding all reachable points
5. Repeat for all unvisited points

### Strengths:
- ✅ Automatically finds number of clusters (no K needed!)
- ✅ Can find arbitrarily-shaped clusters (not just circles)
- ✅ Identifies outliers as noise
- ✅ Robust to noise

### Weaknesses:
- ❌ Struggles with varying densities
- ❌ Sensitive to eps and min_samples parameters
- ❌ Not great for high-dimensional data

### Real AI Use (2024-2025):
- **Anomaly detection:** Noise points = anomalies!
- **Geospatial clustering:** Group locations (rides, deliveries)
- **RAG document clustering:** Find dense topic clusters, ignore outliers
- **Image segmentation:** Group pixels by density

In [ ]:
# Create complex-shaped data (K-Means would fail here!)
X_moons, _ = make_moons(n_samples=300, noise=0.05, random_state=42)

# Add some noise points (outliers)
noise_points = np.random.uniform(low=-2, high=3, size=(20, 2))
X_complex = np.vstack([X_moons, noise_points])

# Try K-Means (will fail on this data!)
kmeans_complex = KMeans(n_clusters=2, random_state=42, n_init=10)
kmeans_labels = kmeans_complex.fit_predict(X_complex)

# Try DBSCAN (will succeed!)
dbscan = DBSCAN(eps=0.3, min_samples=5)
dbscan_labels = dbscan.fit_predict(X_complex)

# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: K-Means (struggles with non-circular shapes)
axes[0].scatter(X_complex[:, 0], X_complex[:, 1], 
               c=kmeans_labels, cmap='viridis', s=50, alpha=0.7)
axes[0].scatter(kmeans_complex.cluster_centers_[:, 0], 
               kmeans_complex.cluster_centers_[:, 1],
               c='red', marker='X', s=300, edgecolors='black', linewidths=2)
axes[0].set_title('❌ K-Means Fails on Complex Shapes', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Feature 1', fontsize=11)
axes[0].set_ylabel('Feature 2', fontsize=11)

# Plot 2: DBSCAN (handles complex shapes!)
# Noise points are labeled as -1
unique_labels = set(dbscan_labels)
colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))

for label, color in zip(unique_labels, colors):
    if label == -1:
        # Noise points in black
        col = 'black'
        marker = 'x'
        label_name = 'Noise'
    else:
        col = color
        marker = 'o'
        label_name = f'Cluster {label}'
    
    class_member_mask = (dbscan_labels == label)
    xy = X_complex[class_member_mask]
    axes[1].scatter(xy[:, 0], xy[:, 1], c=[col], marker=marker,
                   s=50, alpha=0.7, label=label_name, edgecolors='black', linewidths=0.5)

axes[1].set_title('✅ DBSCAN Succeeds!', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Feature 1', fontsize=11)
axes[1].set_ylabel('Feature 2', fontsize=11)
axes[1].legend()

plt.tight_layout()
plt.show()

# Count clusters and noise
n_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise = list(dbscan_labels).count(-1)

print("🎯 DBSCAN Results:")
print(f"   - Found {n_clusters} clusters automatically (no K needed!)")
print(f"   - Identified {n_noise} noise points (outliers)")
print(f"   - Parameters: eps={dbscan.eps}, min_samples={dbscan.min_samples}")
print("\n💡 Key Insight:")
print("   - K-Means assumes circular clusters → fails on crescent shapes")
print("   - DBSCAN uses density → perfectly separates crescents!")
print("   - Bonus: DBSCAN found outliers (black X marks) automatically!")

## 🤖 Real AI Example: RAG Document Clustering

**Scenario:** You're building a RAG system for a company's knowledge base.
- 1000 documents embedded using OpenAI's `text-embedding-3-small` (1536 dimensions)
- Goal: Cluster similar documents to improve retrieval speed

**Why cluster for RAG?**
1. **Faster retrieval:** Search within relevant cluster first
2. **Better organization:** Group docs by topic
3. **Diverse results:** Sample from different clusters
4. **Anomaly detection:** Find unique/outlier documents

Let's simulate this with simplified embeddings!

In [ ]:
# Simulate document embeddings (in reality: 1536-dim vectors from OpenAI)
# We'll use 2D for visualization, but same logic applies to 1536D!

np.random.seed(42)

# Create document embeddings representing different topics
# Cluster 1: HR documents (benefits, policies)
hr_docs = np.random.randn(100, 2) * 0.5 + np.array([2, 2])

# Cluster 2: Engineering docs (APIs, architecture)
eng_docs = np.random.randn(100, 2) * 0.5 + np.array([-2, 2])

# Cluster 3: Sales docs (pricing, contracts)
sales_docs = np.random.randn(100, 2) * 0.5 + np.array([0, -2])

# Cluster 4: Marketing docs (campaigns, analytics)
marketing_docs = np.random.randn(100, 2) * 0.5 + np.array([3, -1])

# Some outlier docs (miscellaneous)
outliers = np.random.randn(20, 2) * 0.3 + np.array([-3, -2])

# Combine all
X_docs = np.vstack([hr_docs, eng_docs, sales_docs, marketing_docs, outliers])
doc_names = (['HR']*100 + ['Engineering']*100 + ['Sales']*100 + 
            ['Marketing']*100 + ['Other']*20)

# Apply K-Means clustering
kmeans_rag = KMeans(n_clusters=5, random_state=42, n_init=10)
cluster_ids = kmeans_rag.fit_predict(X_docs)

# Visualize
plt.figure(figsize=(12, 8))
scatter = plt.scatter(X_docs[:, 0], X_docs[:, 1], 
                     c=cluster_ids, cmap='tab10', s=50, alpha=0.6)
plt.scatter(kmeans_rag.cluster_centers_[:, 0], 
           kmeans_rag.cluster_centers_[:, 1],
           c='red', marker='★', s=500, edgecolors='black', linewidths=2,
           label='Cluster Centers', zorder=5)
plt.title('RAG Document Clustering: Knowledge Base Organization', 
         fontsize=16, fontweight='bold')
plt.xlabel('Embedding Dimension 1', fontsize=12)
plt.ylabel('Embedding Dimension 2', fontsize=12)
plt.colorbar(scatter, label='Cluster ID')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.show()

# Analyze clusters
print("📚 RAG DOCUMENT CLUSTERING RESULTS")
print("=" * 60)
for cluster_id in range(5):
    cluster_mask = (cluster_ids == cluster_id)
    cluster_size = np.sum(cluster_mask)
    print(f"\n🗂️  Cluster {cluster_id}: {cluster_size} documents")
    
    # Show distribution of doc types in this cluster
    cluster_docs = np.array(doc_names)[cluster_mask]
    unique, counts = np.unique(cluster_docs, return_counts=True)
    for doc_type, count in zip(unique, counts):
        percentage = (count / cluster_size) * 100
        print(f"   - {doc_type}: {count} docs ({percentage:.1f}%)")

print("\n" + "=" * 60)
print("\n🎯 How This Improves RAG Performance:")
print("\n1️⃣  Faster Retrieval:")
print("   - Query: 'What are our vacation policies?'")
print("   - Instead of searching 420 docs → search HR cluster only (~100 docs)")
print("   - 4x speed improvement!")
print("\n2️⃣  Diverse Results:")
print("   - Sample top-k from different clusters")
print("   - Avoid retrieving 5 very similar documents")
print("\n3️⃣  Better Organization:")
print("   - Create topic-based indices in vector database")
print("   - Route queries to relevant clusters first")
print("\n4️⃣  Anomaly Detection:")
print("   - Small clusters or noise points = unique/outlier docs")
print("   - May need special handling or better categorization")

## 📊 Clustering Evaluation Metrics

**Challenge:** How do we measure clustering quality without labels?

### Internal Metrics (no labels needed):

**1. Silhouette Score**
- Measures how similar a point is to its own cluster vs. other clusters
- Range: -1 to 1
  - 1 = Perfect (far from other clusters)
  - 0 = On the boundary
  - -1 = Wrong cluster
- Formula: `(b - a) / max(a, b)` where:
  - a = average distance to points in same cluster
  - b = average distance to points in nearest other cluster

**2. Davies-Bouldin Index**
- Measures average similarity between clusters
- Lower is better (0 = perfect)
- Compares within-cluster scatter to between-cluster separation

**3. Inertia (K-Means specific)**
- Sum of squared distances to nearest centroid
- Lower is better, but always decreases with more clusters
- Use elbow method to find sweet spot

In [ ]:
# Evaluate our customer segmentation clustering
silhouette_avg = silhouette_score(X_customers, cluster_labels)
davies_bouldin = davies_bouldin_score(X_customers, cluster_labels)

print("📊 CLUSTERING QUALITY METRICS")
print("=" * 60)
print(f"\n✅ Silhouette Score: {silhouette_avg:.3f}")
print("   Interpretation:")
if silhouette_avg > 0.7:
    print("   🌟 Excellent! Strong, well-separated clusters")
elif silhouette_avg > 0.5:
    print("   ✅ Good! Reasonable cluster structure")
elif silhouette_avg > 0.25:
    print("   ⚠️  Fair. Clusters exist but may overlap")
else:
    print("   ❌ Poor. Clusters are weak or artificial")

print(f"\n📉 Davies-Bouldin Index: {davies_bouldin:.3f}")
print("   Interpretation:")
if davies_bouldin < 0.5:
    print("   🌟 Excellent! Tight, well-separated clusters")
elif davies_bouldin < 1.0:
    print("   ✅ Good cluster separation")
else:
    print("   ⚠️  Clusters may be overlapping")

print(f"\n🎯 Inertia: {kmeans.inertia_:.2f}")
print("   (Lower is better, but use elbow method for comparison)")

print("\n" + "=" * 60)
print("\n💡 Key Insight:")
print("   These metrics help you compare different clustering algorithms")
print("   or different values of K, even without ground truth labels!")

## 🎯 YOUR TURN: Customer Segmentation Challenge

You work for an e-commerce company. You have data on customer behavior:
- Feature 1: Average order value (dollars)
- Feature 2: Number of purchases per month
- Feature 3: Days since last purchase

**Your task:**
1. Generate sample customer data
2. Use K-Means to find customer segments
3. Use the elbow method to find optimal K
4. Evaluate clustering quality
5. Interpret the segments for marketing strategy

In [ ]:
# YOUR TURN: Complete this exercise!

# Step 1: Generate customer data (3 features)
X_ecommerce, _ = make_blobs(
    n_samples=500,
    n_features=3,
    centers=# YOUR CODE - try different values!,
    cluster_std=# YOUR CODE - control how spread out clusters are,
    random_state=42
)

# Step 2: Apply K-Means (start with K=3)
kmeans_ecom = KMeans(
    n_clusters=# YOUR CODE,
    random_state=42,
    n_init=10
)
labels_ecom = # YOUR CODE - fit and predict

# Step 3: Find optimal K using elbow method
inertias_ecom = []
k_range = range(1, 10)

for k in k_range:
    # YOUR CODE - fit K-Means with different K values
    # Append inertia to list
    pass

# Plot elbow curve
plt.figure(figsize=(10, 6))
# YOUR CODE - create elbow plot
plt.show()

# Step 4: Evaluate best clustering
best_k = # YOUR CODE - choose based on elbow
kmeans_best = KMeans(n_clusters=best_k, random_state=42, n_init=10)
labels_best = kmeans_best.fit_predict(X_ecommerce)

silhouette = # YOUR CODE
davies_bouldin = # YOUR CODE

print(f"Best K: {best_k}")
print(f"Silhouette Score: {silhouette:.3f}")
print(f"Davies-Bouldin Index: {davies_bouldin:.3f}")

# Step 5: Interpret segments
print("\n🎯 CUSTOMER SEGMENTS:")
for i in range(best_k):
    cluster_data = X_ecommerce[labels_best == i]
    print(f"\nSegment {i}: {len(cluster_data)} customers")
    print(f"  Avg Order Value: ${cluster_data[:, 0].mean():.2f}")
    print(f"  Purchases/Month: {cluster_data[:, 1].mean():.1f}")
    print(f"  Days Since Last: {cluster_data[:, 2].mean():.1f}")
    # YOUR CODE - add interpretation (e.g., "VIP customers", "At-risk", etc.)

### ✅ Solution

In [ ]:
# SOLUTION: Customer Segmentation

# Step 1: Generate customer data
X_ecommerce, _ = make_blobs(
    n_samples=500,
    n_features=3,
    centers=4,  # 4 natural segments
    cluster_std=1.0,  # Moderate spread
    random_state=42
)

# Make data more realistic (positive values, different scales)
X_ecommerce[:, 0] = X_ecommerce[:, 0] * 20 + 100  # Avg order: $80-120
X_ecommerce[:, 1] = np.abs(X_ecommerce[:, 1] * 2 + 5)  # Purchases: 1-10/month
X_ecommerce[:, 2] = np.abs(X_ecommerce[:, 2] * 5 + 15)  # Days since: 5-30

# Step 2: Find optimal K
inertias_ecom = []
silhouette_scores_ecom = []
k_range = range(2, 10)

for k in k_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X_ecommerce)
    inertias_ecom.append(km.inertia_)
    silhouette_scores_ecom.append(silhouette_score(X_ecommerce, km.labels_))

# Plot elbow
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(k_range, inertias_ecom, marker='o', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Clusters (K)', fontsize=12)
axes[0].set_ylabel('Inertia', fontsize=12)
axes[0].set_title('Elbow Method: E-commerce Customer Segments', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)
axes[0].axvline(x=4, color='red', linestyle='--', alpha=0.7, label='Optimal K=4')
axes[0].legend()

axes[1].plot(k_range, silhouette_scores_ecom, marker='s', color='green', linewidth=2, markersize=8)
axes[1].set_xlabel('Number of Clusters (K)', fontsize=12)
axes[1].set_ylabel('Silhouette Score', fontsize=12)
axes[1].set_title('Silhouette Score by K', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)
axes[1].axhline(y=0.5, color='orange', linestyle='--', alpha=0.7, label='Good threshold')
axes[1].legend()

plt.tight_layout()
plt.show()

# Step 3: Fit best model
best_k = 4
kmeans_best = KMeans(n_clusters=best_k, random_state=42, n_init=10)
labels_best = kmeans_best.fit_predict(X_ecommerce)

silhouette = silhouette_score(X_ecommerce, labels_best)
davies_bouldin = davies_bouldin_score(X_ecommerce, labels_best)

print("📊 CLUSTERING EVALUATION")
print("=" * 60)
print(f"Best K: {best_k}")
print(f"Silhouette Score: {silhouette:.3f} - {'✅ Good!' if silhouette > 0.5 else '⚠️ Fair'}")
print(f"Davies-Bouldin Index: {davies_bouldin:.3f} - {'✅ Good!' if davies_bouldin < 1.0 else '⚠️ Fair'}")

# Step 4: Interpret segments
segment_names = []
print("\n🎯 CUSTOMER SEGMENTS & MARKETING STRATEGY")
print("=" * 60)

for i in range(best_k):
    cluster_data = X_ecommerce[labels_best == i]
    avg_order = cluster_data[:, 0].mean()
    purchases_month = cluster_data[:, 1].mean()
    days_since = cluster_data[:, 2].mean()
    
    # Classify segment based on behavior
    if avg_order > 110 and purchases_month > 6:
        name = "💎 VIP Customers"
        strategy = "Loyalty rewards, early access to sales, personalized service"
    elif purchases_month > 6:
        name = "🔥 Frequent Buyers"
        strategy = "Upsell premium products, subscription offers"
    elif avg_order > 110:
        name = "👑 High-Value Buyers"
        strategy = "Encourage repeat purchases with targeted campaigns"
    elif days_since > 20:
        name = "⚠️  At-Risk/Dormant"
        strategy = "Re-engagement campaigns, discount codes, 'we miss you' emails"
    else:
        name = "🆕 Casual Shoppers"
        strategy = "Nurture with content, build brand loyalty, first-time buyer discounts"
    
    print(f"\n{name}")
    print(f"  📊 Size: {len(cluster_data)} customers ({len(cluster_data)/len(X_ecommerce)*100:.1f}%)")
    print(f"  💰 Avg Order Value: ${avg_order:.2f}")
    print(f"  📦 Purchases/Month: {purchases_month:.1f}")
    print(f"  📅 Days Since Last: {days_since:.1f}")
    print(f"  🎯 Strategy: {strategy}")

print("\n" + "=" * 60)
print("\n💡 Business Value:")
print("   ✅ Personalized marketing campaigns for each segment")
print("   ✅ Efficient resource allocation (focus on high-value)")
print("   ✅ Identify at-risk customers for retention efforts")
print("   ✅ Optimize product recommendations per segment")
print("   ✅ Predict lifetime value and churn risk")

## 📋 Algorithm Comparison Cheat Sheet

| Feature | K-Means | Hierarchical | DBSCAN |
|---------|---------|--------------|--------|
| **Need to specify K?** | ✅ Yes | ✅ Yes | ❌ No |
| **Cluster shape** | Spherical only | Any | Any |
| **Handles noise/outliers** | ❌ No | ❌ No | ✅ Yes |
| **Scalability** | 🟢 Fast (millions) | 🔴 Slow (thousands) | 🟡 Medium |
| **Deterministic** | ❌ No (random init) | ✅ Yes | ✅ Yes |
| **Memory usage** | 🟢 Low | 🔴 High | 🟡 Medium |
| **Best for** | Large data, spherical | Small data, hierarchy | Complex shapes, noise |

**Decision tree:**
1. **Data > 100K points?** → K-Means
2. **Need hierarchy?** → Hierarchical
3. **Complex shapes or noise?** → DBSCAN
4. **Unknown K, simple shapes?** → K-Means + Elbow method

## 🎉 Congratulations!

**You just mastered:**
- ✅ K-Means clustering and elbow method
- ✅ Hierarchical clustering and dendrograms
- ✅ DBSCAN for density-based clustering and outlier detection
- ✅ Clustering evaluation metrics (Silhouette, Davies-Bouldin)
- ✅ Real AI applications: RAG document clustering, customer segmentation
- ✅ When to use which algorithm

**🎯 Key Takeaways:**
1. **K-Means** = Fast, simple, needs K, assumes spherical clusters
2. **Hierarchical** = Visual tree, no K needed initially, slow on large data
3. **DBSCAN** = Finds any shape, detects outliers, no K needed
4. **Evaluation** = Use Silhouette & Davies-Bouldin for quality measurement
5. **Real AI** = RAG systems use clustering for faster retrieval!

**🚀 Practice Challenge (Before Day 2!):**

Apply clustering to a real dataset:
1. Load a dataset (e.g., Iris, customer data, or text embeddings)
2. Try all three algorithms (K-Means, Hierarchical, DBSCAN)
3. Compare results using evaluation metrics
4. Visualize and interpret the clusters
5. Which algorithm works best? Why?

---

**📚 Next Lesson:** Day 2 - Dimensionality Reduction (PCA, t-SNE, UMAP for visualizing high-dimensional embeddings!)

**💬 Questions?** Try clustering different shapes of data - you'll see when each algorithm shines!

---

*"In unsupervised learning, the patterns you find are often more interesting than the patterns you expected!"* 🎯